ライブラリのインポート
必要なデータを取得し、画像データをnp配列train_Xへ格納
ラベルデータもtrain_yへ格納
DLモデルを構築(今回はLeNet(初期の基礎的なCNNモデル)を学習させる)


In [135]:
import numpy as np
import pandas as pd
from PIL import Image
import os

In [146]:
#ラベルデータの読み込み
train_master = pd.read_csv("train_master.tsv", sep="\t")
train_master.head()

,file_name,category_id
0,train_0.jpg,5
1,train_1.jpg,0
2,train_2.jpg,4
3,train_3.jpg,1
4,train_4.jpg,9


In [137]:
#画像データ
image = Image.open("./train_images/train_0.jpg")
print('width, height:', image.size)
image

width, height: (28, 28)


In [138]:

train_master["category_id"].value_counts().sort_index()

category_id
0    5923
1    6742
2    5958
3    6131
4    5842
5    5421
6    5918
7    6265
8    5851
9    5949
Name: count, dtype: int64

In [139]:
num_train = len(train_master)
image_size = (28,28)

In [140]:
#train_Xは4次元の配列で初期値はすべて0(6000枚*1色* 28*28ピクセル)
#ここにすべての画像データを1ピクセルごとに輝度を数値化して格納していく
train_X = np.zeros( (num_train,) + (1,) + image_size)

#train_yは画像データのラベルデータ(目的変数)
train_y = np.zeros((num_train,))

In [141]:
image_folder = "./train_images"
image_files = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder) ]

In [147]:
#全ての画像とラベルを変換して配列に格納
for data in train_master.iterrows(): 
    #iterrowsで(data[0]==index, data[1]==[画像データ名,ラベル(答え)])をタプルで取得する。
    image = Image.open(image_files[data[0]])
    image_array = np.array(image).reshape( (1,) + image_size)
    train_X[data[0], :] = image_array / 255.
    train_y[data[0]] = data[1]["category_id"]


In [ ]:
#モデル計算の効率を上げるため。多くのニューラルネットワークライブラリ（Chainer, PyTorch, TensorFlowなど）は float32 を推奨。
train_X = train_X.astype(np.float32)
train_y = train_y.astype(np.int32)


In [144]:
#DLモデルの構築。今回はLeNet
import torch
import torch.nn as nn
import torch.nn.functional as F

In [150]:
#pytorch使うときはtensor型へ
train_X = torch.tensor(train_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.int32)

In [ ]:
#ニューラルネットワークLeNetのクラスを定義
class LeNet(nn.Module):
    def __init__(self, out_size=10):
        super(LeNet, self).__init__()
        # 畳み込み層
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1)
        # 全結合層
        self.fc3 = nn.Linear(16 * 5 * 5, 120)  # 入力サイズ: 特徴マップサイズ (5x5) * チャネル数 (16)
        self.fc4 = nn.Linear(120, 84)
        self.fc5 = nn.Linear(84, out_size)
        
        # 活性化関数を設定
        self.act_func = F.relu
       

    def forward(self, x):
        # 順伝播
        x = F.max_pool2d(self.act_func(self.conv1(x)), kernel_size=2, stride=2)  # 畳み込み1 + プーリング
        x = F.max_pool2d(self.act_func(self.conv2(x)), kernel_size=2, stride=2)  # 畳み込み2 + プーリング
        x = torch.flatten(x, 1)  # 平坦化
        x = self.act_func(self.fc3(x))  # 全結合1
        x = self.act_func(self.fc4(x))  # 全結合2
        x = self.fc5(x)  # 出力層
        return x
